# Santa 2021 - The Merry Movie Montage

People seem to be getting in the Christmas spirit earlier and earlier each year. Decorations appear for sale in stores in the fall, Christmas songs are on the radio in October…

The Elves at the North Pole are starting to recognize this, and need to work as fast as possible to launch their latest holiday offering: SantaTV+! A 24/7 streaming television channel where it’s “Always Christmas, All the Time.” To debut their new station, they’ve decided to kick things off with a made-for-television Christmas movie marathon! They’re excited for the premiere of such movies as 🎅, 🤶, 🦌, 🧝, 🎄, 🎁, and 🎀!

But elves know that just as important as the movie themselves is the order they’ll be aired. So the elves have decided the best way to figure out which order is best is to watch all the movies in every possible combination to see which feels the most Christmas-y.

Your job is to help the elves by giving them the shortest viewing schedules that shows them every combination of movies so they can get SantaTV+ live as soon as possible! The elves have formed three movie-watching teams to lighten the load, so every combination must be seen by at least one of their groups. But they’re also pretty sure they want to kick off the movie marathon with the 🎅 and 🤶 movies back-to-back, so be sure that each group has all the combinations that start with those. And finally, the elves have agreed to two sugar breaks, so you’re allowed to give each group up to two 🌟 wildcards, which will play all the movies at once while they’re snacking, which will help speed things along.

They can’t launch SantaTV+ until all the groups have finished watching - so help give them the most efficient schedule to see every Christmas movie combination, and help them get back to making toys!

https://www.kaggle.com/c/santa-2021/overview

In [1]:
!conda info


     active environment : kaggle-santa2021
    active env location : C:\ProgramData\Anaconda3\envs\kaggle-santa2021
            shell level : 2
       user config file : C:\Users\globetrekker\.condarc
 populated config files : C:\Users\globetrekker\.condarc
          conda version : 4.10.3
    conda-build version : 3.21.4
         python version : 3.8.8.final.0
       virtual packages : __win=0=0
                          __archspec=1=x86_64
       base environment : C:\ProgramData\Anaconda3  (writable)
      conda av data dir : C:\ProgramData\Anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/m

In [2]:
import csv
import copy
import numpy as np
import pandas as pd
import os
import itertools

from datetime import datetime

%config Completer.use_jedi = False

## Common functions

In [14]:
def get_movies(env: str):
    if env == 'test':
        movies = ['1', '2', '3', '4']
    else:
        movies = ['🎅', '🤶', '🦌', '🧝', '🎄', '🎁', '🎀']
        
    return movies

In [16]:
def get_perms(env: str, movies: list, path: str) -> list:
    """Given list of movies, list of permutations"""
    
    if env == 'test':
        perm_raw = list(itertools.permutations(movies))
        perms = []
        for p in perm_raw:
            perm = ""
            for item in p:
                perm += str(item)
            perms.append(perm)
    else:
        with open(path, newline='', encoding="utf-8") as f:
            reader = csv.reader(f)
            p = list(reader)

        p = p[1:]
        perms = [a[0] for a in p]
    
    return perms

In [20]:
def get_perm_nos(perms: list) -> list:
    """Given of permutations, return perm numbers"""
    
    perm_nos = []
    for i in range(len(perms)):
        perm_nos.append(i)
        
    return perm_nos

In [22]:
def get_distance_matrix(env: str, perms: list, perm_nos: list, path: str):
    """Given list of permutations, return distance matrix"""
    
    if env == 'test':
        idx = list(perms)
        cols = list(perms)
        df_dist = pd.DataFrame(columns=cols, index=idx)
        for r in list(df_dist.index):
            for c in list(df_dist.columns):
                if str(r)[1:] == str(c)[:3]:
                    df_dist.loc[r, c] = 1
                elif str(r)[2:] == str(c)[:2]:
                    df_dist.loc[r, c] = 2
                elif str(r)[3:] == str(c)[:1]:
                    df_dist.loc[r, c] = 3
                else:
                    df_dist.loc[r, c] = 4
    else:
        df_dist = pd.read_csv(path)
        df_dist.set_index('Permutation', drop=True, inplace=True)

    df_dist_nos = copy.deepcopy(df_dist)
    cols = perm_nos
    idx = perm_nos
    df_dist_nos.columns = cols
    df_dist_nos.index = cols
    
    
    return df_dist, df_dist_nos

In [23]:
def get_distance_list(df_dist_nos: pd.DataFrame, dist: int):
    """Given distance matrix (nos), return distance list"""
    
    distance_list = []
    for a in range(1, dist):
        for col in df_dist_nos.columns:
            idx = df_dist_nos.index[df_dist_nos[col] == a].tolist()
            if len(idx) > 0:
                for i in idx:
                    row = [a, i, col]
                    distance_list.append(row)
                    
    return distance_list

In [24]:
def get_mandatory_perms(perms: list, movies: list):
    
    mandatory_perms = [p for p in perms if p[0]==movies[0] and p[1]==movies[1]]
    mandatory_perm_nos = []
    for p in mandatory_perms:
        mandatory_perm_nos.append(perm_nos[perms.index(p)])
    
    return mandatory_perms, mandatory_perm_nos

In [8]:
def init(num_strs, perm_nos, mandatory_perm_nos):
    """Initialize"""
    strs = []
    rem_mand_perms = [] # remaining list of mandatory perms
    str_dists = [] # list of strings

    rem_perms = [p for p in perm_nos if p not in mandatory_perm_nos]

    for i in range(num_strs):
        new_str = [mandatory_perm_nos[i]]
        new_mand_perms = copy.deepcopy(mandatory_perm_nos)
        new_mand_perms.remove(mandatory_perm_nos[i])

        rem_mand_perms.append(new_mand_perms)
        strs.append([new_str])
        str_dists.append([])
    
    return strs, str_dists, rem_perms, rem_mand_perms

## Main code

In [47]:
env = 'prod'
movies = get_movies(env)
perms = get_perms(env, movies, '../data/permutations.csv')
perm_nos = get_perm_nos(perms)
df_dist, df_dist_nos = get_distance_matrix(env, perms, perm_nos, '../data/distance_matrix.csv')
distance_list = get_distance_list(df_dist_nos, len(movies)+1)
mandatory_perms, mandatory_perm_nos = get_mandatory_perms(perms, movies)
distance_list_mandatory = [d for d in distance_list if d[2] in mandatory_perm_nos]
distance_list_nonmandatory = [d for d in distance_list if d[2] not in mandatory_perm_nos]
strs, str_dists, rem_perms, rem_mand_perms = init(3, perm_nos, mandatory_perm_nos)

## Common

**Algorithm**
1. 
  - for each string  
  - check if any mandatory perms can be found with dist 1  
  - if yes, add perm, add distance, reset dist and restart from 1
  - if no, continue
2. 
  - identify string with shortest distance
  - check if any non-mandatory perms can be found with current dist
  - if yes, add perm, add distance, reset dist and restart from 1
  - if no, increment dist, restart from 1

**Notes**
1. Any time any perm is added, dist is reset and loop is restarted
2. Non-mandatory perms are added to string with smallest distance

In [48]:
def algo_v1(strs, str_dists, rem_perms, rem_mand_perms):
    """Version 1 of algorithm to minimize length of strings"""
    
    dist = 1
    print(len(rem_perms))
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)

    ctr = 1
    max_ctr = -1
    while len(rem_perms) > 0 and (ctr <= max_ctr or max_ctr == -1):
        ctr += 1
        if len(rem_perms) % 100 == 0:
            print(len(rem_perms))
            now = datetime.now()

            current_time = now.strftime("%H:%M:%S")
            print("Current Time =", current_time)

        # attempt to find d1 remaining mandatory perms in each list
        loop1 = True
        while loop1:
            loop1 = False
            for i in range(len(strs)):        
                trail = strs[i][0][-1]
                list_match = [p for p in distance_list_mandatory 
                              if p[0]==1 \
                              and p[1] == trail \
                              and p[2] in rem_mand_perms[i]]

                if len(list_match) > 0:
                    strs[i][0].append(list_match[0][2])
                    rem_mand_perms[i].remove(list_match[0][2])
                    trail = strs[i][0][-1]
                    str_dists[i].append(dist)
                    loop1 = True
                    if dist > 1:
                        dist = 1


        # get string with shortest length
        sum_dist = 10000000
        shortest_i = -1
        for i in range(len(strs)):
            if sum(str_dists[i]) < sum_dist:
                sum_dist = sum(str_dists[i])
                shortest_i = i
        i = shortest_i

        # check if any perms can be found with dist
        trail = strs[i][0][-1]
        list_match = [p for p in distance_list_nonmandatory 
                      if p[0]==dist \
                      and p[1] == trail \
                      and p[2] in rem_perms]
        if list_match:
            strs[i][0].append(list_match[0][2])
            rem_perms.remove(list_match[0][2])
            str_dists[i].append(dist)
            if dist > 1:
                dist = 1
            continue
        else:
            dist += 1

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("End Time =", current_time)
    
    return strs, str_dists, rem_perms

In [49]:
def algo_v2(strs, str_dists, rem_perms, rem_mand_perms):
    """Version 2 of algorithm to minimize length of strings"""
    
    dist = 1
    print(len(rem_perms))
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)

    ctr = 1
    max_ctr = -1
    while len(rem_perms) > 0 and (ctr <= max_ctr or max_ctr == -1):
        ctr += 1
        if len(rem_perms) % 100 == 0:
            print(len(rem_perms))
            now = datetime.now()

            current_time = now.strftime("%H:%M:%S")
            print("Current Time =", current_time)

        # attempt to find d1 remaining mandatory perms in each list
        loop1 = True
        while loop1:
            loop1 = False
            for i in range(len(strs)):        
                trail = strs[i][0][-1]
                list_match = [p for p in distance_list_mandatory 
                              if p[0]==dist \
                              and p[1] == trail \
                              and p[2] in rem_mand_perms[i]]

                if list_match:
                    strs[i][0].append(list_match[0][2])
                    rem_mand_perms[i].remove(list_match[0][2])
                    trail = strs[i][0][-1]
                    str_dists[i].append(dist)
                    loop1 = True
                    if dist > 1:
                        dist = 1


        # get string lengths
        s_lengths = []
        for i in range(len(strs)):
            s_lengths.append((i, sum(str_dists[i])))
        s_lengths.sort(key=lambda x:x[1])
        
        # if difference between shortest and longest strings is > 7, 
        # only attempt to find match in shortest string, to even things out
        # if not consider all strings
        strs_i = []
        if len(s_lengths) == 3:
            if abs(s_lengths[2][1] - s_lengths[0][1]) >= 7:
                strs_i.append(s_lengths[0][0])
        else:
            for i in range(len(strs)):
                strs_i.append(i)
                
        continue_non_mand_perm_search = True # to make sure addition of 1 perm at a time
        for i in strs_i:
            if continue_non_mand_perm_search:
                # check if any perms can be found with dist
                trail = strs[i][0][-1]
                list_match = [p for p in distance_list_nonmandatory 
                              if p[0]==dist \
                              and p[1] == trail \
                              and p[2] in rem_perms]
                if list_match:
                    strs[i][0].append(list_match[0][2])
                    rem_perms.remove(list_match[0][2])
                    str_dists[i].append(dist)
                    continue_non_mand_perm_search = False
                    if dist > 1:
                        dist = 1
                    continue
                else:
                    dist += 1

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("End Time =", current_time)
    
    return strs, str_dists, rem_perms

In [ ]:
strs, str_dists, rem_perms, rem_mand_perms = init(3, perm_nos, mandatory_perm_nos)
strs, str_dists, rem_perms = algo_v2(strs, str_dists, rem_perms, rem_mand_perms)

4920
Start Time = 01:58:01


In [18]:
rem_perms

[3324]

In [ ]:
data = strs[0][0]
file = 'algo_v2.csv'

with open(file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)
    
data = strs[1][0]

with open(file, 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)
    
data = strs[2][0]

with open(file, 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)

In [ ]:
data = str_dists[0]
file = 'algo_dist_v2.csv'

with open(file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)
    
data = str_dists[1]

with open(file, 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)
    
data = str_dists[2]

with open(file, 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(data)

In [19]:
p_nm = [p for p in perm_nos if p not in mandatory_perm_nos]
p_nm_np = [p for p in p_nm if (p not in strs[0][0]) and (p not in strs[1][0]) and (p not in strs[2][0])]
p_m = [p for p in perm_nos if p in mandatory_perm_nos]
p_m_np1 = [p for p in p_m if p not in strs[0][0]]
p_m_np2 = [p for p in p_m if p not in strs[1][0]]
p_m_np3 = [p for p in p_m if p not in strs[2][0]]

print('string 1 length: %d' % (sum(str_dists[0])))
print('string 2 length: %d' % (sum(str_dists[1])))
print('string 3 length: %d' % (sum(str_dists[2])))

print('nonmandatory perms %d' % (len(p_nm)))
print('nonmandatory perms not in any string: %d' % (len(p_nm_np)))
print('mandatory perms %d' % (len(p_m)))
print('mandatory perms not in s1: %d' % (len(p_m_np1)))
print('mandatory perms not in s2: %d' % (len(p_m_np2)))
print('mandatory perms not in s3: %d' % (len(p_m_np3)))

string 1 length: 2588
string 2 length: 2590
string 3 length: 2171
nonmandatory perms 4920
nonmandatory perms not in any string: 1
mandatory perms 120
mandatory perms not in s1: 0
mandatory perms not in s2: 16
mandatory perms not in s3: 65


In [ ]:
# algo_v1

# string 1 length: 1679
# string 2 length: 1679
# string 3 length: 1679
# total distance: 5037
# nonmandatory perms 4920
# nonmandatory perms not in any string: 0
# mandatory perms 120
# mandatory perms not in s1: 83
# mandatory perms not in s2: 79
# mandatory perms not in s3: 78

In [ ]:
# algo_v2

# string 1 length: 2645
# string 2 length: 2181
# string 3 length: 2137
# nonmandatory perms 4920
# nonmandatory perms not in any string: 0
# mandatory perms 120
# mandatory perms not in s1: 0
# mandatory perms not in s2: 74
# mandatory perms not in s3: 60